# De Scikit a Tensorflow

L'objectiu d'aquest quadern és realitzar la transició entre el Multi-Layer perceptron de Scikit amb el qual vàrem treballar la setmana passada i la llibreria  TensorFlow.

Avui treballarem amb xarxes on totes les neurones de cada capa es troben connectades amb les neurones de la capa següent, tal com està dissenyada la classe MLP de Scikit. 

Començarem usant els mateixos conjunts de dades de la darrera sessió i intentarem reproduïr resultats similars.

In [ ]:
# Llibreries 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons, make_circles, make_classification

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# TensorFlow
import tensorflow as tf
from tensorflow import keras


## Dades


Generació dels conjunts de dades ja coneguts, en aquest cas tendrem conjunts de dades amb 300 mostres.

In [ ]:
# Construïm 4 conjunts de dades per classificar:
# En primer lloc un conjunt linealment separable on afegim renou
X, y = make_classification(n_samples=300, n_features=2, n_redundant=0, n_informative=2, random_state=33, n_clusters_per_class=1)
rng = np.random.RandomState(2)
X += 2 * rng.uniform(size=X.shape)
linearly_separable = (X, y)

# En segon lloc un que segueix una distribució xor
X_xor = np.random.randn(300, 2)
y_xor = np.logical_xor(X_xor[:, 0] > 0,
                       X_xor[:, 1] > 0)
y_xor = np.where(y_xor, 1, -1)

# Els afegim a una llista juntament amb els seus noms per tal de poder iterar
# sobre ells
datasets = [
    ("linear", linearly_separable),
    ("moons", make_moons(n_samples=300, noise=0.3, random_state=30)),  # Tercer dataset
    ("circles", make_circles(n_samples=300, noise=0.2, factor=0.5, random_state=30)),  # Darrer dataset
    ("xor", (X_xor, y_xor))]

## Model

Tensorflow ens permet definir l'arquitectura de les xarxes de dues maneres, en aquest cas emprarem l'entorn seqüencial. Deixarem l'entorn funcional per a futures pràctiques. 

Especificam les diferents capes com a part d'un objecte de la classe model.

Les capes que emprarem es diuen denses `Dense` ([enllaç a la documentació](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense)). 

Emprarem 2 paràmetres:
 - El nombre de neurones.
 - La funció d'activació.



In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input((2)),
    tf.keras.layers.Dense(2, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
# En alguns tutorials veureu la definició de la xarxa així:
#model = tf.keras.Sequential()
#model.add(tf.keras.layers.Input((2)))
#model.add(tf.keras.layers.Dense(5, activation='relu'))
#model.add(tf.keras.layers.Dense(1, activation='sigmoid'))


## Compilar el model

El model que em definit necessita ser compilat, en aquesta passa es defineix l'algorisme d'optimització, la funció de pèrdua i les mètriques sobre les quals volem obtenir resultats.

Funcions de pèrdua:
- Documentació [funcions de perdua](https://www.tensorflow.org/api_docs/python/tf/keras/losses?version=nightly)

- Petit [tutorial](https://www.analyticsvidhya.com/blog/2021/05/guide-for-loss-function-in-tensorflow/)

In [ ]:
model.compile(optimizer='adam', # també podria ser el descens de gradient tradicional
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
# Ens proporciona un resum de com és la nostra xarxa, fixau-vos en el nombre de
# paràmetres
model.summary()

## Entrenament

El procés d'entrenament és molt similar a qualsevol altre mètode d'aprenentatge automàtic. Podem observar 2 diferències:

- Hem d'explicitar el nombre d'èpoques (nombre d'iteracions de l'entrenament).
- Podem aportar un conjunt de dades d'avaluació per tenir una estimació de la bondat del nostre model al final de cada època.

Finalment veureu que la funció fit ens proporciona un diccionari (variable `history`), emprarem les dades guardades en ell per poder dibuixar els gràfics de resum l'entrenament.

In [ ]:
# En primer lloc preparam les dades

X, y = datasets[1][1]

X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=33)

In [ ]:
# En segon lloc entrenam:
# - Proporcionam les dades d'entrenament i de validació
# - Definim el nombre d'iteracions
# - Es pot definir el tamany del batch (batch_size)
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15)
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)

print('\nTest accuracy:', test_acc)

In [ ]:
# (history.history.keys())

# Mostram els resultats de l'entrenament de manera gràfica
figure, ax = plt.subplots(nrows=1, ncols=2)

ax[0].plot(history.history['accuracy'])
ax[0].plot(history.history['val_accuracy'])

ax[0].set_title('model accuracy')
ax[0].set_ylabel('accuracy')
ax[0].set_xlabel('epoch')
ax[0].set_ylim(0,1)
ax[0].legend(['train', 'test'], loc='upper left')

# summarize history for loss
ax[1].plot(history.history['loss'])
ax[1].plot(history.history['val_loss'])
ax[1].set_ylim(0,1)
ax[1].set_title('model loss')
ax[1].set_ylabel('loss')
ax[1].set_xlabel('epoch')
ax[1].legend(['train', 'test'], loc='upper right')
figure.tight_layout()
plt.show()

## Predicció

In [ ]:
result = model.predict(X)

print(result)